In [ ]:
# Project 1
# Volatility Breakout Strategy

#### 2022-2학기
#### Version: 2022-11-11-v1

> 분반:
>
> 학번:
>
> 학과:
>
> 이름:




In [ ]:
# 런타임이 종료되면 이 코드셀은 매번 다시 실행해야 합니다. 译：runtime结束后，这个coad cell每次都要重新运行。
# pyupbit 설치 译：设置pyupbit
!pip install pyupbit

In [ ]:
# Import

import numpy as np
import pandas as pd
import datetime

import pyupbit


In [ ]:
# trading_helper 설치 및 버전 확인 译：# trading_helper 设置和版本确认

!wget https://esohn.be/python/trading_helper.py -O trading_helper.py

import trading_helper
import importlib
importlib.reload(trading_helper)

print( f'Trading Helper Version: {trading_helper.TRADING_HELPER_VERSION}'  )

In [ ]:
# ------------------------------------------------------------------------------------------------------
# https://esohn.be/account 의 AUTH CODE 입력 译：https://esohn.be/account 的 AUTH CODE 输入
# ------------------------------------------------------------------------------------------------------
'''
AUTH_CODE = 'f587ea651febd5446efc61d5e3fef123'  -> 아래에 이렇게 auth code를 붙여넣으세요. 译：请在下面这样粘贴auth code
'''
AUTH_CODE = '2319d129e17e2cd365363ef4ebfb4596'

# Do not edit
trading_helper.set_auth_code( AUTH_CODE )


### 아래 코드셀을 복사해서 프로젝트 코드로 붙여 넣으세요.
### 译：复制下面的代码单元并粘贴到项目代码中。

In [9]:
# ------------------------------------------------------------------------------------------------------
# - 코드를 완성했다면 이 코드셀만 복사해서 competition 페이지 Volatility Breakout 탭에 라운드 별 코드 항목으로 붙여넣습니다.
# - 연습할 때는 https://esohn.be/competition 페이지의 에디터에서 직접 연습하지 말고 Colab에서 연습하세요. 
#   (노트북 연습 횟수가 서버에 기록되며 또 노트북 자체에 변경 history가 축척 됩니다.)
#译：
#完成代码后，只需复制此代码单元（code cell）并将其作为每轮代码条目粘贴到competition页面的“Volatility Breakout”中。
#练习的时候不要在https://esohn.be/competition页面的编辑, 请在Colab里练习
#（笔记本电脑练习次数将被记录在服务器上，笔记本电脑本身也会积累变更history。）
# ------------------------------------------------------------------------------------------------------

# Given Code Version 2022.11.10-v1


import numpy as np
import pandas as pd
import datetime


# [프로젝트 1]
# volatility_breakout() 변동성 돌파 전략 함수 
#
# df - 일별 가격 정보를 가지고 있는 데이터 프레임
# k - k값, 기본값은 0.5
#
# Qualification 라운드 에서는 상승장 구분을 사용하지 않음 (df['cond2'] = True로 남겨둠)
# 그 외의 라운드에서는 원하는 k값과 상승장 구분을 사용하여 경쟁

#译：[项目1]
#    volatility_breakout() 变动性突破战略函数
#    df - 具有每日价格信息的数据框架
#    k - k 值， 默认值（基本值）为 0.5
#    Qualification 回合中不使用 上升场（bull market？） 分类 (df['cond2'] = 保留为True)
#
def volatility_breakout( df, k=0.5 ):

    if k < 0:
        return
        
    # 전일 레인지 range_s = (고가 - 저가).shift(1)
    #译：前一天的range  range_s = (高价 - 低价).shift(1)
    df['range_s'] = (df['high'] - df['low']).shift(1) # EDIT (1)

    # 목표 매수가 = 오늘 시가 + 전일 레인지 * k
    #译：目标买入 = 今日市价 + 前一天的range * k
    df['target'] = df['open'] + df['range_s']*k # EDIT (2)

    # 먄약 이동평균을 활용하고 싶으면 다음과 같이 이동평균 열을 추가
    #译：如果想使用moving average，请添加以下 moving average 列
    # df['ma5'] = df['close'].rolling(5).mean().shift(1)
    # df['ma20'] = df['close'].rolling(20).mean().shift(1)
    # df['ma50'] = df['close'].rolling(50).mean().shift(1)

    #--------------------------------------------------------------
    # 조건 1 - 변동성 돌파 ( np.where( 고가 >= 목표 매수가, True, False ) )
    #译：条件 1 - 突破变动性 (np.where(高价 > = 目标收购, True, False))
    '''
    cond1 에는 변동성 돌파 전략을 위한 수식만 사용 가능합니다.
    译：cond1只能使用用于可变性突破策略的公式。
    '''

    df['cond1'] = False # EDIT (3)
    #--------------------------------------------------------------

    #--------------------------------------------------------------
    # 조건 2 - 상승장 구분 (optional)
    #译：条件 2 - 区分上升场 (optional)
    #
    # 추가 조건이 없다면, 译：如果没有追加的条件的话
    #   df['cond2'] = True 
    # 5일 이동평균보다 시가가 높을 때만 매매한다면, 译：如果只在市价高于5日移动平均值（moving average）时进行交易，
    #   df['cond2'] = np.where( (df['open'] >= df['ma5']), True, False )
    # 3일 이동평균 및 5일 이동평균보다 시가가 높을 때만 매매한다면, 译：如果仅在市价高于3天移动平均值和5天移动平均值时进行交易，
    #   df['cond2'] = np.where( (df['open'] >= df['ma3']) & (df['open'] >= df['ma5']), True, False )
    # 
    '''
    cond2 에서 open 이외에 오늘 결정되지 않은 미래의 정보(close, high, low, volume)를 그냥 사용하면 안됩니다.
    译：在cond2中,除了open之外,不能直接使用今天没有决定的未来信息(close, high, low, volume)。
    .shift(1) 를 붙이면 모두 사용 가능합니다.
    译：粘贴了shift(1)即都可使用。

    예를 들어,（ 比如）
    - df['open'] : 사용 가능（可以使用） 
    - df['close'].shift(1) : 사용 가능（可以使用）
    - df['high'] : 사용 불가능（不可以使用）
    - df['low'].shift(1) : 사용 가능（可以使用）
    - df['close'].shift(2) : 사용 가능（可以使用）
    '''
    
    df['cond2'] = True # EDITABLE
    print(df)
    #--------------------------------------------------------------

    return df

# 이 코드 셀은 함수 정의 이외에는 다른 내용을 아래에 덧붙이지 마세요.
#译：这个code cell（代码单元）除了函数定义以外，不要在下面添加其他内容。
# ------------------------------------------------------------------------------------------------------


### 가격정보 가져오기（获取价格信息）

In [7]:
# 가격정보 가져오기 （获取价格信息）
from_date = '20210401'
to_date = '20221101'

df = trading_helper.get_ohlcv2('KRW-BTC', date1=from_date, date2=to_date )

df.head()


,Unnamed: 0,open,high,low,close,volume,value
0,2021-04-01 09:00:00+09:00,71480000.0,73325000.0,71032000.0,72749000.0,9170.381958,6.630544e+11
1,2021-04-02 09:00:00+09:00,72749000.0,74400000.0,71841000.0,73855000.0,8712.294082,6.413909e+11
2,2021-04-03 09:00:00+09:00,73934000.0,75703000.0,71783000.0,72187000.0,9274.318109,6.892717e+11
3,2021-04-04 09:00:00+09:00,72187000.0,74240000.0,71570000.0,73974000.0,6583.438203,4.821920e+11
4,2021-04-05 09:00:00+09:00,73908000.0,78780000.0,73890000.0,78770000.0,13425.681678,1.019472e+12


### k=0.5, 0.8일 때의 HPR, MDD （k=0.5、0.8时的HPR、MDD）

In [4]:
# 함수 테스트 （函数测试）
volatility_breakout( df, 0.5 )

hpr, mdd = trading_helper.check_performance_vol( df )

print( f'HPR = {hpr * 100:.1f}%, MDD = {mdd * 100:.1f}%' )

volatility_breakout( df, 0.8 )

hpr, mdd = trading_helper.check_performance_vol( df )

print( f'HPR = {hpr * 100:.1f}%, MDD = {mdd * 100:.1f}%' )

NameError: name 'volatility_breakout' is not defined

### HPR 그래프 （HPR图）

In [10]:
# 기간 수익률(HPR) 변화 추이 그래프 （# 期收益率(HPR)变化趋势图）

( (df['hpr']-1) * 100).plot( figsize=(15,3) )

KeyError: 'hpr'

### MDD 그래프 （MDD图）

In [ ]:
# 낙폭(Drawdown) 변화 추이 그래프 （# 降幅变化趋势图）

(-df['dd'] * 100).plot( figsize=(15,3) )

### 여러 개의 k값을 시도해보고 싶을 때 （想要尝试多个k值时）

In [ ]:
# 여러개의 k값을 시도해보고 싶을 때 （想要尝试多个k值时）

records = [];

for k in np.arange(0.4, 0.5, 0.02):

    volatility_breakout( df, k )

    hpr, mdd = trading_helper.check_performance_vol( df )

    records.append( ( hpr * 100, mdd * 100, k) )

records.sort(reverse=True)

for record in records:
    print(f'HPR= {record[0]:.1f}% MDD= {record[1]:.1f}% k= {record[2]:.2f}')
